<a href="https://colab.research.google.com/github/nd823/data-cleaning/blob/master/telco_data_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Introduction

Good quality and consistently formated data is absolutely key for any successful data analysis project. In this ongoing series, I will create a personal knowledge base of various aspects of data cleaning and transformation.

Where applicable, I will use the Telco data set as the example, though other data sets will appear as I demonstrate new techniques.

Let's get started!

# 2. Import data

We will get the dataset in its raw form:

In [1]:
import pandas as pd

df = pd.read_csv("https://github.com/treselle-systems/customer_churn_analysis/raw/master/WA_Fn-UseC_-Telco-Customer-Churn.csv")

# 3. Intial check

Recall from our preliminary exploratory data analysis using the `dataMaid` package in a [previous post](https://kyso.io/nd823/exploratory-data-analysis/file/1.%20automated_EDA.ipynb) that there are several issues with this data set that need to be fixed:

- The `customerID` column needs to be removed
- Headers of the `gender` and `tenure` columns are not capitalized, while those of the other columns are
- The `TotalCharges` column has some missing values and at least one row has `Tenure`=0
    - It is quite possible that the rows where `Tenure`=0 are also the rows where `TotalCharges` values are missing, as it would be impossible to calculate the latter without a positive value of the former
- The `SeniorCitizen` column is of `integer` type as it is encoded in 0s and 1s, while other categorical features are encoded in strings as the `object` type
- Several categorical features have three levels (e.g. 'Yes'/'No'/'No internet service') where the 'No internet service' level contains very few data points and can be consolidated into 'No'

We take a quick look at the data and column properties to confirm that these are indeed in need of addressing:

In [2]:
df.head().T

,0,1,2,3,4
customerID,7590-VHVEG,5575-GNVDE,3668-QPYBK,7795-CFOCW,9237-HQITU
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
customerID          7043 non-null object
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null object
Churn               7043 non-null object
dtypes: float64(1), int64(2), obj

Additionally, we see that the `TotalCharges` column has the wrong data type (`object` rather than `float64`), which is also masking the missing values.

Let's starting cleaning!

# 4. Data cleaning

## 4.1 Drop `customerID` column

In [4]:
df = df.drop(['customerID'], axis = 1)

## 4.2 Rename columns for consistent capitalization

In [5]:
df.rename(columns={'gender': 'Gender', 'tenure':'Tenure'}, inplace=True)

## 4.3 Remove rows with `Tenure`=0 and missing `TotalCharges` values

We first convert `TotalCharges` to the right column data type, `float64`:

In [6]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
Gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
Tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7032 non-null float64
Churn               7043 non-null object
dtypes: float64(2), int64(2), object(16)
memory usage: 1.1+ MB


Now we see that indeed there are 11 values missing in the `TotalCharges` column.

Let's check if these 11 rows also contain `Tenure`=0:

In [7]:
## Get rows where Tenure=0 OR TotalCharges value is missing
df[(df['Tenure'] == 0) | (df['TotalCharges'].isnull())]

,Gender,SeniorCitizen,Partner,Dependents,Tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
488,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,NaN,No
753,Male,0,No,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,NaN,No
936,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,NaN,No
1082,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,NaN,No
1340,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,NaN,No
3331,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,19.85,NaN,No
3826,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,NaN,No
4380,Female,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,NaN,No
5218,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,NaN,No
6670,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,NaN,No


Indeed, the two sets of problematic data points are the same. As there are only 11 such data points out of >7,000 in total, we can remove them.

In [8]:
df = df[df['Tenure'] > 0 ]

df.shape

(7032, 20)

## 4.4 Re-encode `SeniorCitizen` column

For the sake of consistency with the other categorical features, we will encode the `SeniorCitizen` column in "Yes"/"No":

In [9]:
import numpy as np

df['SeniorCitizen'] = df['SeniorCitizen'].astype(str)
                       
df['SeniorCitizen'] = np.where(df['SeniorCitizen']=='1', 'Yes', 'No')

## 4.5 Combine sparse levels

Also, we will consolidate the 'No internet service' levels into 'No' to reduce cardinality:

In [10]:
for col in ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']:
    df[col] = df[col].replace({'No internet service':'No'})
    
df['MultipleLines'] = df['MultipleLines'].replace({'No phone service':'No'})

# 5. Final check

In [11]:
df.head().T

,0,1,2,3,4
Gender,Female,Male,Male,Male,Female
SeniorCitizen,No,No,No,No,No
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
Tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No,No,No,No,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No
OnlineBackup,Yes,No,Yes,No,No


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 20 columns):
Gender              7032 non-null object
SeniorCitizen       7032 non-null object
Partner             7032 non-null object
Dependents          7032 non-null object
Tenure              7032 non-null int64
PhoneService        7032 non-null object
MultipleLines       7032 non-null object
InternetService     7032 non-null object
OnlineSecurity      7032 non-null object
OnlineBackup        7032 non-null object
DeviceProtection    7032 non-null object
TechSupport         7032 non-null object
StreamingTV         7032 non-null object
StreamingMovies     7032 non-null object
Contract            7032 non-null object
PaperlessBilling    7032 non-null object
PaymentMethod       7032 non-null object
MonthlyCharges      7032 non-null float64
TotalCharges        7032 non-null float64
Churn               7032 non-null object
dtypes: float64(2), int64(1), object(17)
memory usage: 1.1+ MB


We quickly check that the appropriate category levels have been converted to "Yes"/"No" encoding:

In [13]:
for col in df.columns:
    if col not in ['Tenure', 'MonthlyCharges', 'TotalCharges']:
        print(f"{col}: {df[col].unique()}")

Gender: ['Female' 'Male']
SeniorCitizen: ['No' 'Yes']
Partner: ['Yes' 'No']
Dependents: ['No' 'Yes']
PhoneService: ['No' 'Yes']
MultipleLines: ['No' 'Yes']
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: ['No' 'Yes']
OnlineBackup: ['Yes' 'No']
DeviceProtection: ['No' 'Yes']
TechSupport: ['No' 'Yes']
StreamingTV: ['No' 'Yes']
StreamingMovies: ['No' 'Yes']
Contract: ['Month-to-month' 'One year' 'Two year']
PaperlessBilling: ['Yes' 'No']
PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
Churn: ['No' 'Yes']


# 6. Export to CSV file

Here I export the cleaned data set to CSV for later use. 

## 6.1 Retain "Yes"/"No" encoding of categorical variables

This form of the dataframe is useful for survival analysis and feature engineering for machine learning.

In [14]:
df.to_csv('./telco_cleaned_yes_no.csv', index=False)

## 6.2 Rename category levels to reflect variable name

To prepare the dataframe for [factor analysis](https://kyso.io/nd823/famd/file/FAMD_part_3.ipynb), we will rename levels of all categorical variables to reflect the column name.

In [15]:
df_renamed = df.copy()

col_list = ['SeniorCitizen', 'Partner', 'Dependents','PhoneService', 'DeviceProtection', 'MultipleLines', 'OnlineSecurity', 
            'OnlineBackup', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn']

for col in col_list:
    df_renamed[col] = np.where(df_renamed[col]=='Yes', col, 'No'+' '+col)
    
df_renamed['InternetService'] = df_renamed['InternetService'].replace({'No':'No internet service'})

In [16]:
df_renamed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7032 entries, 0 to 7042
Data columns (total 20 columns):
Gender              7032 non-null object
SeniorCitizen       7032 non-null object
Partner             7032 non-null object
Dependents          7032 non-null object
Tenure              7032 non-null int64
PhoneService        7032 non-null object
MultipleLines       7032 non-null object
InternetService     7032 non-null object
OnlineSecurity      7032 non-null object
OnlineBackup        7032 non-null object
DeviceProtection    7032 non-null object
TechSupport         7032 non-null object
StreamingTV         7032 non-null object
StreamingMovies     7032 non-null object
Contract            7032 non-null object
PaperlessBilling    7032 non-null object
PaymentMethod       7032 non-null object
MonthlyCharges      7032 non-null float64
TotalCharges        7032 non-null float64
Churn               7032 non-null object
dtypes: float64(2), int64(1), object(17)
memory usage: 1.1+ MB


In [17]:
df_renamed.head().T

,0,1,2,3,4
Gender,Female,Male,Male,Male,Female
SeniorCitizen,No SeniorCitizen,No SeniorCitizen,No SeniorCitizen,No SeniorCitizen,No SeniorCitizen
Partner,Partner,No Partner,No Partner,No Partner,No Partner
Dependents,No Dependents,No Dependents,No Dependents,No Dependents,No Dependents
Tenure,1,34,2,45,2
PhoneService,No PhoneService,PhoneService,PhoneService,No PhoneService,PhoneService
MultipleLines,No MultipleLines,No MultipleLines,No MultipleLines,No MultipleLines,No MultipleLines
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No OnlineSecurity,OnlineSecurity,OnlineSecurity,OnlineSecurity,No OnlineSecurity
OnlineBackup,OnlineBackup,No OnlineBackup,OnlineBackup,No OnlineBackup,No OnlineBackup


A quick check that all is well with the category level names:

In [18]:
for col in df_renamed.columns:
    if col not in ['Tenure', 'MonthlyCharges', 'TotalCharges']:
        print(f"{col}: {df_renamed[col].unique()}")

Gender: ['Female' 'Male']
SeniorCitizen: ['No SeniorCitizen' 'SeniorCitizen']
Partner: ['Partner' 'No Partner']
Dependents: ['No Dependents' 'Dependents']
PhoneService: ['No PhoneService' 'PhoneService']
MultipleLines: ['No MultipleLines' 'MultipleLines']
InternetService: ['DSL' 'Fiber optic' 'No internet service']
OnlineSecurity: ['No OnlineSecurity' 'OnlineSecurity']
OnlineBackup: ['OnlineBackup' 'No OnlineBackup']
DeviceProtection: ['No DeviceProtection' 'DeviceProtection']
TechSupport: ['No TechSupport' 'TechSupport']
StreamingTV: ['No StreamingTV' 'StreamingTV']
StreamingMovies: ['No StreamingMovies' 'StreamingMovies']
Contract: ['Month-to-month' 'One year' 'Two year']
PaperlessBilling: ['PaperlessBilling' 'No PaperlessBilling']
PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
Churn: ['No Churn' 'Churn']


In [19]:
df_renamed.to_csv('./telco_cleaned_renamed.csv', index=False)

Any comments or suggestions for improvements will be greatly appreciated! 

Til next time! :)